# Modelagem Preditiva — Score de Risco Operacional de Internações

## Interpretabilidade de internações de ALTO RISCO

O objetivo desta etapa foi desenvolver um modelo preditivo de apoio à decisão, capaz de priorizar internações hospitalares que apresentam maior risco operacional, considerando aspectos assistenciais, regulatórios e financeiros.

No contexto da Gestão de Internações Hospitalares (GIH), a priorização eficiente permite direcionar esforços de auditoria, acompanhamento clínico e gestão financeira para os casos com maior potencial de impacto, aumentando a eficiência operacional e reduzindo riscos.

In [30]:
# =========================
# Setup e configurações
# =========================
import os
os.chdir(r'C:\Projetos\case-ami-saude')

import pandas as pd
import numpy as np
import re
from scipy import stats

from src.utils import pipeline_universal_limpeza , cid_formato_valido , cid_compativel_especialidade
from src.utils import OutlierConfig, find_outliers, filter_outlier_rows, plot_outlier_boxplots, boxplot_iqr_todas_colunas

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report , confusion_matrix
from sklearn.impute import SimpleImputer

import joblib

In [2]:
path_base_01 = r'C:\Projetos\case-ami-saude\data\raw\Base 1 – Internações Hospitalares.csv'
path_base_02 = r'C:\Projetos\case-ami-saude\data\raw\Base 2 – Itens da Internação.csv'
df_base_01 = pd.read_csv(path_base_01)
df_base_02 =pd.read_csv(path_base_02)

In [14]:
df1 = df_base_01.copy()
df2 = df_base_02.copy()

# --- Datas: coerce para evitar quebra com valores ruins
date_cols_df1 = [
    "data_nascimento",
    "data_solicitacao_autorizacao",
    "data_autorizacao_senha",
    "data_admissao",
    "data_alta",
]
for c in date_cols_df1:
    if c in df1.columns:
        df1[c] = pd.to_datetime(df1[c], errors="coerce")

date_cols_df2 = ["data_item"]
for c in date_cols_df2:
    if c in df2.columns:
        df2[c] = pd.to_datetime(df2[c], errors="coerce")

# --- Numericas: coerce para garantir float/int
num_cols_df1 = ["idade", "tempo_autorizacao_horas", "valor_total_conta", "valor_pago"]
for c in num_cols_df1:
    if c in df1.columns:
        df1[c] = pd.to_numeric(df1[c], errors="coerce")

num_cols_df2 = ["quantidade_solicitada", "quantidade_autorizada", "valor_unitario", "valor_total_item", "valor_glosado"]
for c in num_cols_df2:
    if c in df2.columns:
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

In [15]:
df1["dias_internado"] = (df1["data_alta"] - df1["data_admissao"]).dt.days

# sanity checks (úteis pro EDA também)
qtd_datas_invalidas = df1[["data_admissao", "data_alta"]].isna().any(axis=1).sum()
qtd_dias_negativos = (df1["dias_internado"] < 0).sum()

print("Linhas com data_admissao ou data_alta nulas:", qtd_datas_invalidas)
print("Linhas com dias_internado negativo:", qtd_dias_negativos)

Linhas com data_admissao ou data_alta nulas: 1200
Linhas com dias_internado negativo: 15


In [16]:
df1.loc[df1["dias_internado"] < 0, "dias_internado"] = np.nan

In [17]:
# Flags de glosa por item: padronizar para 0/1
# (caso venha como "Sim/Não", "1/0", True/False, etc.)
def to_flag01(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip().str.upper()
    mapping = {"SIM": 1, "S": 1, "TRUE": 1, "1": 1,
               "NÃO": 0, "NAO": 0, "N": 0, "FALSE": 0, "0": 0}
    out = s.map(mapping)
    return out.astype("Int64")

if "glosa_item_flag" in df2.columns:
    df2["glosa_item_flag_01"] = to_flag01(df2["glosa_item_flag"])
else:
    df2["glosa_item_flag_01"] = pd.Series(pd.NA, index=df2.index, dtype="Int64")

# Agregações principais
agg_itens = df2.groupby("senha_internacao", as_index=False).agg(
    itens_qtd=("item_id", "count"),
    itens_distintos=("codigo_item", pd.Series.nunique),
    valor_itens_total=("valor_total_item", "sum"),
    valor_glosado_total=("valor_glosado", "sum"),
    qtd_itens_glosados=("glosa_item_flag_01", "sum"),
)

# Features derivadas
agg_itens["pct_itens_glosados"] = agg_itens["qtd_itens_glosados"] / agg_itens["itens_qtd"]
agg_itens["pct_valor_glosado"] = agg_itens["valor_glosado_total"] / agg_itens["valor_itens_total"]

# proteção contra divisão por zero / NaN
agg_itens.replace([np.inf, -np.inf], np.nan, inplace=True)

agg_itens.head()

,senha_internacao,itens_qtd,itens_distintos,valor_itens_total,valor_glosado_total,qtd_itens_glosados,pct_itens_glosados,pct_valor_glosado
0,SI20250000001,20,9,105888.66,0.00,0,0.0,0.000000
1,SI20250000002,27,12,303439.62,54161.45,4,0.148148,0.178492
2,SI20250000003,13,8,35417.69,0.00,0,0.0,0.000000
3,SI20250000004,18,8,24660.48,1236.87,2,0.111111,0.050156
4,SI20250000005,21,11,39977.75,6493.91,4,0.190476,0.162438


In [18]:
df = df1.merge(agg_itens, on="senha_internacao", how="left")

# Onde não tem itens (internações sem registro na base 02), zera agregados
fill_zero_cols = ["itens_qtd", "itens_distintos", "valor_itens_total", "valor_glosado_total", "qtd_itens_glosados"]
for c in fill_zero_cols:
    df[c] = df[c].fillna(0)

# Percentuais podem ficar NaN se valor_itens_total = 0
df["pct_itens_glosados"] = df["pct_itens_glosados"].fillna(0)
df["pct_valor_glosado"] = df["pct_valor_glosado"].fillna(0)

df.shape

(20000, 44)

In [19]:
p90_dias = df["dias_internado"].quantile(0.90)
p90_custo = df["valor_total_conta"].quantile(0.90)

df["target_risco"] = (
    (df["glosa_flag"].astype("string").str.upper().isin(["1", "SIM", "TRUE"])) |
    (df["dias_internado"] > p90_dias) |
    (df["valor_total_conta"] > p90_custo)
).astype(int)

df["target_risco"].value_counts(normalize=True).round(3)

target_risco
0    0.599
1    0.401
Name: proportion, dtype: float64

In [20]:
p90_pct_glosa = df["pct_valor_glosado"].quantile(0.90)
df["target_glosa_alta"] = (df["pct_valor_glosado"] > p90_pct_glosa).astype(int)

df["target_glosa_alta"].value_counts(normalize=True).round(3)

target_glosa_alta
0    0.9
1    0.1
Name: proportion, dtype: float64

In [ ]:
features_num = [
    "idade",
    "tempo_autorizacao_horas",
    "itens_qtd",
    "itens_distintos",
    "pct_itens_glosados",
]

features_cat = [
    "perfil_hospital",
    "tipo_plano",
    "segmentacao_plano",
    "acomodacao",
    "carater_internacao",
    "tipo_internacao",
    "especialidade_responsavel",
    "complexidade",
    "empresa_auditoria",
    "status_regulacao",
    "auditoria_responsavel",
]

features_flag_cat = [
    "uti_flag",
    "suporte_ventilatorio_flag",
    "hemodialise_flag",
]

features_cat = [c for c in features_cat if c in df.columns]
features_flag_cat = [c for c in features_flag_cat if c in df.columns]
features_num = [c for c in features_num if c in df.columns]

features = features_num + features_cat + features_flag_cat

X = df[features].copy()
y = df["target_risco"].astype(int)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, features_num),
        ("cat", cat_pipe, features_cat + features_flag_cat),
    ],
    remainder="drop"
)

clf = LogisticRegression(
    max_iter=3000,
    class_weight="balanced"
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", clf),
])

model.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [26]:
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

print("AUC:", roc_auc_score(y_test, y_prob))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

AUC: 0.9649509522822959

Classification report:
               precision    recall  f1-score   support

           0       0.91      0.96      0.93      3592
           1       0.93      0.86      0.89      2408

    accuracy                           0.92      6000
   macro avg       0.92      0.91      0.91      6000
weighted avg       0.92      0.92      0.92      6000


Confusion matrix:
 [[3436  156]
 [ 345 2063]]


In [27]:
df_scores = df[["senha_internacao"]].copy()
df_scores["prob_risco"] = model.predict_proba(X)[:, 1]
df_scores["score_prioridade"] = (df_scores["prob_risco"] * 100).round(1)

df_scores["classe_risco"] = pd.cut(
    df_scores["score_prioridade"],
    bins=[0, 40, 70, 100],
    labels=["Baixo", "Médio", "Alto"],
    include_lowest=True
)

df_scores.head()

,senha_internacao,prob_risco,score_prioridade,classe_risco
0,SI20250000001,0.039012,3.9,Baixo
1,SI20250000002,1.000000,100.0,Alto
2,SI20250000003,0.022677,2.3,Baixo
3,SI20250000004,0.999983,100.0,Alto
4,SI20250000005,1.000000,100.0,Alto


In [28]:
output_path = "data/processed/score_risco_internacoes.csv"
df_scores.to_csv(output_path, index=False)
output_path

'data/processed/score_risco_internacoes.csv'

In [32]:
joblib.dump(model, "models/model_risco_internacao.joblib")
print("Modelo salvo em models/model_risco_internacao.joblib")


Modelo salvo em models/model_risco_internacao.joblib
